In [1]:
#1.log level counter

f = open("applicaiton_logs.txt","r")

data = f.read()

words = data.split()
freq = {}

for word in words:
    freq[word] = freq.get(word,0)+1
    
f.close()
    
print("The frequency of each word:\n",freq)

The frequency of each word:
 {'2025-03-01': 5, '14:05:00,001': 1, 'INFO': 2, 'ModuleA': 3, '-': 5, 'User': 1, 'login': 1, 'successful': 1, 'for': 5, 'user': 2, "'alice'.": 1, '14:05:05,002': 1, 'DEBUG': 1, 'ModuleB': 1, 'Retrieved': 1, 'data': 1, "'alice'": 1, 'from': 1, 'cache.': 1, '14:05:10,003': 1, 'ERROR': 1, 'ModuleC': 1, 'Payment': 3, 'processing': 2, 'failed': 1, 'transaction': 3, 'ID': 3, '12345.': 3, '14:05:15,004': 1, 'WARNING': 1, 'retry': 2, 'scheduled': 1, '14:05:20,005': 1, 'succeeded': 1, 'on': 1}


In [2]:
#2.Error Log Extractor

f = open("applicaiton_logs.txt",'r')


log = []

while True:
    data = f.readline()
    if not data:    
        break
    data = data.strip()
    if not data:      
        continue
    parts = data.split(" ",3)
    timestamp = parts[0] + ' ' + parts[1]
    severity = parts[2]
    _ ,message = parts[3].split("-",1)
    
    log.append({"timestamp":timestamp,"severity":severity,"message":message})

f.close()
    
for logs in log:
    print(logs)

{'timestamp': '2025-03-01 14:05:00,001', 'severity': 'INFO', 'message': " User login successful for user 'alice'."}
{'timestamp': '2025-03-01 14:05:05,002', 'severity': 'DEBUG', 'message': " Retrieved user data for 'alice' from cache."}
{'timestamp': '2025-03-01 14:05:10,003', 'severity': 'ERROR', 'message': ' Payment processing failed for transaction ID 12345.'}
{'timestamp': '2025-03-01 14:05:15,004', 'severity': 'WARNING', 'message': ' Payment retry scheduled for transaction ID 12345.'}
{'timestamp': '2025-03-01 14:05:20,005', 'severity': 'INFO', 'message': ' Payment processing succeeded on retry for transaction ID 12345.'}


In [3]:
#Log Parser to Dictionary
def func():
    f = open("applicaiton_logs.txt","r")

    data = f.read()

    words = data.split()
    freq = {}

    for word in words:
        freq[word] = freq.get(word,0)+1
    
    f.close()
    
    print("The frequency of each word:\n",freq)

func()

The frequency of each word:
 {'2025-03-01': 5, '14:05:00,001': 1, 'INFO': 2, 'ModuleA': 3, '-': 5, 'User': 1, 'login': 1, 'successful': 1, 'for': 5, 'user': 2, "'alice'.": 1, '14:05:05,002': 1, 'DEBUG': 1, 'ModuleB': 1, 'Retrieved': 1, 'data': 1, "'alice'": 1, 'from': 1, 'cache.': 1, '14:05:10,003': 1, 'ERROR': 1, 'ModuleC': 1, 'Payment': 3, 'processing': 2, 'failed': 1, 'transaction': 3, 'ID': 3, '12345.': 3, '14:05:15,004': 1, 'WARNING': 1, 'retry': 2, 'scheduled': 1, '14:05:20,005': 1, 'succeeded': 1, 'on': 1}


In [4]:
#Group Logs by Module
severity_counts = {}

with open("large_log.txt", "r") as f:
    for line in f:
        parts = line.split()
        if len(parts) > 2:
            severity = parts[2].replace(":", "")
            if severity in severity_counts:
                severity_counts[severity] += 1
            else:
                severity_counts[severity] = 1

print(severity_counts)

{'INFO': 28, 'DEBUG': 19, 'WARNING': 3, 'ERROR': 1}


In [8]:
#Format Validator with Exception Handling
import re

log_pattern = re.compile(
    r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3} (INFO|DEBUG|ERROR|WARNING): .+$"
)

def validate_log_file(file_path):
    try:
        with open(file_path, "r") as f:
            for line_number, line in enumerate(f, start=1):
                line = line.strip()
                try:
                    if not log_pattern.match(line):
                        print(f"Warning: Line {line_number} is misformatted -> {line}")
                except Exception as e:
                    print(f"❌ Error processing line {line_number}: {e}")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Unexpected error while reading file: {e}")

if __name__ == "__main__":
    log_file = "mixed_logs.log"  
    validate_log_file(log_file)
    print("Validation complete.")


Validation complete.


In [15]:
#JSON Log Converter
import re
import json

# Expected log format: "2025-03-01 10:00:00,100 INFO: [Start] - Batch processing started."
log_pattern = re.compile(
    r"^(?P<date>\d{4}-\d{2}-\d{2}) "
    r"(?P<time>\d{2}:\d{2}:\d{2},\d{3}) "
    r"(?P<level>INFO|DEBUG|ERROR|WARNING): "
    r"(?P<message>.+)$"
)

def log_to_json(input_file, output_file):
    logs = []
    try:
        with open(input_file, "r") as f:
            for line_number, line in enumerate(f, start=1):
                line = line.strip()
                match = log_pattern.match(line)
                if match:
                    logs.append(match.groupdict())  
                else:
                    logs.append({
                        "line_number": line_number,
                        "error": "misformatted",
                        "raw": line
                    })
        with open(output_file, "w") as out:
            json.dump(logs, out, indent=4)
        print(f"Logs converted to JSON and saved in '{output_file}'")
    except FileNotFoundError:
        print(f"File not found: {input_file}")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    log_to_json("applicaiton_logs.txt", "logs.json")


Logs converted to JSON and saved in 'logs.json'


In [16]:
#Command-Line Log Filter

msg = "ERROR"
with open("large_log.txt",'r') as f:
    lsts = [x for x in f if msg not in x]
    print(lsts)
    
    

['2025-03-01 10:00:00,100 INFO: [Start] - Batch processing started.\n', '2025-03-01 10:00:05,150 DEBUG: [Process] - Step 1 initiated.\n', '2025-03-01 10:00:10,200 INFO: [Process] - Step 1 completed.\n', '2025-03-01 10:00:15,250 DEBUG: [Process] - Step 2 initiated.\n', '2025-03-01 10:00:20,300 INFO: [Process] - Step 2 completed.\n', '2025-03-01 10:00:25,350 DEBUG: [Process] - Step 3 initiated.\n', '2025-03-01 10:00:30,400 INFO: [Process] - Step 3 completed.\n', '2025-03-01 10:00:35,450 WARNING: [Process] - Slow response detected.\n', '2025-03-01 10:00:40,500 INFO: [Process] - Retrying step 3.\n', '2025-03-01 10:00:45,550 INFO: [Process] - Step 3 completed after retry.\n', '2025-03-01 10:00:50,600 DEBUG: [Monitor] - Checking system status.\n', '2025-03-01 10:00:55,650 INFO: [Monitor] - System status nominal.\n', '2025-03-01 10:01:00,700 DEBUG: [Data] - Loading batch 1.\n', '2025-03-01 10:01:05,750 INFO: [Data] - Batch 1 loaded.\n', '2025-03-01 10:01:10,800 DEBUG: [Data] - Processing batc

In [14]:
#Regular Expression Challenge(IP Address Extractor)

import re
f = open("sample_log.txt",'r')

pattern = re.compile(
    r"\b(?:25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(?:\.(?:25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3}\b"
)


data = f.read()

match = re.findall(pattern,data)

print(match)

['192.168.1.10', '10.0.0.2', '172.16.0.5', '203.0.113.58', '192.168.1.13', '10.0.0.7', '172.16.0.19', '198.51.100.17']
